In [ ]:
exit()

# We re-perform an explorative data analysis
# profile = ProfileReport(df, title='Titanic raw data analysis', explorative=True)
# # profile.to_widgets() # gross overview
# profile.to_file('./analysis/pandas-analysis-2.html')

# Fare has the highest correlation with survival rate, but has high cardinality, so we categorize
fare_bins = [0, 10, 20, 50]
fare_names = ['<10', '10-20', '20-50', '50+']
fare_d = dict(enumerate(fare_names, 1))
df['Fare'] = np.vectorize(fare_d.get)(np.digitize(df['Fare'], fare_bins))

# We fill missing age values with mean of other columns
df['Age'].fillna(df['Age'].dropna().mean(), inplace=True)

# We fill missing of embarked randomly, there is no significance
df['Embarked'].fillna('Q', inplace=True)

# We categorize age to find a better significance
age_bins = [0, 10, 20, 30, 40,50]
age_names = ['<10', '10-20', '20-30', '30-40', '40-50', '50+']
age_d = dict(enumerate(age_names, 1))
# 0 => infant <2
# 1 => toddler <4
# 2 => child <12
# 3 => teen <18
# 4 => young adult <25
# 5 => adult <35
# 6 => senior <55
# 7 => old >55
# df['Age'] = np.vectorize(age_d.get)(np.digitize(df['Age'], age_bins))

# Convert data
df['Sex'] = df['Sex'].astype(int)
df['SibSp'] = df['SibSp'].astype(int)
df['Parch'] = df['Parch'].astype(int)

df.reset_index(drop=True, inplace=True)

dd = pd.DataFrame({'x': df['Age'], 'data': df['Survived']})
# sns.regplot(x=df['Age'],y=df['Survived'], fit_reg=False)
plt.bar(df['Age'], df['Survived'])
plt.show()
exit()

# We re-perform an explorative data analysis
# profile = ProfileReport(df, title='Titanic raw data analysis 3', explorative=True)
# profile.to_widgets() # gross overview
# profile.to_file('./analysis/pandas-analysis-3.html')

# Start training
X = pd.DataFrame(df.drop(['Survived','SibSp','Parch','Embarked'], axis=True).values, columns=['Pclass', 'Sex', 'Age', 'Ticket', 'Fare'])
y = pd.Series(df['Survived'].values, name='Survived')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)
print(X)
automl = AutoML(algorithms=['Decision Tree', 'Linear', 'Random Forest','Extra Trees','CatBoost','Neural Network','Nearest Neighbors'],total_time_limit=5*60)
automl.fit(X_train, y_train)

y_predicted = automl.predict(X_test)
score = accuracy_score(y_test, y_predicted["label"].astype(int))

print(pd.DataFrame({'Predicted': y_predicted['label'], 'Survived': np.array(y_test), 'Score': score}))
print("Test MSE:", mean_squared_error(y_test, y_predicted["label"]))